# Exercise 6 Solutions
This notebook contains solutions to the week 6 exercises.

We use built-in images from `skimage.data` as the original test images are not included in the repository.

In [ ]:
import numpy as np
import cv2
from skimage import data, color
import matplotlib.pyplot as plt
from scipy.ndimage import maximum_filter


## Exercise 6.1
Implement a Gaussian kernel `g` and its derivative `gd` for a given width $\sigma$. The kernel is truncated at $\pm3\sigma$, which retains more than 99\% of the Gaussian mass. The kernel is normalized to sum to one.

In [ ]:
def gaussian1DKernel(sigma, truncate=3):
    """Return 1D Gaussian kernel and its derivative."""
    if sigma <= 0:
        raise ValueError("sigma must be positive")
    half = int(np.ceil(truncate * sigma))
    x = np.arange(-half, half + 1)
    g = np.exp(-(x**2) / (2 * sigma**2))
    g = g / g.sum()
    gd = -x / (sigma**2) * g
    return g.astype(np.float32), gd.astype(np.float32)

# Example
g, gd = gaussian1DKernel(1.5)
len(g), g.sum()


## Exercise 6.2
Using separable convolution, smooth the image and compute derivatives in $x$ and $y$.

In [ ]:
def gaussianSmoothing(im, sigma):
    im_gray = color.rgb2gray(im) if im.ndim == 3 else im
    im_gray = im_gray.astype(np.float32)
    if sigma == 0:
        return im_gray, np.zeros_like(im_gray), np.zeros_like(im_gray)
    g, gd = gaussian1DKernel(sigma)
    I = cv2.sepFilter2D(im_gray, -1, g, g)
    Ix = cv2.sepFilter2D(im_gray, -1, gd, g)
    Iy = cv2.sepFilter2D(im_gray, -1, g, gd)
    return I, Ix, Iy

# Test on checkerboard image
test_im = data.checkerboard()
I, Ix, Iy = gaussianSmoothing(test_im, 1.5)
fig, axs = plt.subplots(1,3, figsize=(12,4))
for ax, img, title in zip(axs, [I, Ix, Iy], ['I','Ix','Iy']):
    ax.imshow(img, cmap='gray')
    ax.set_title(title)
    ax.axis('off')
plt.show()


## Exercise 6.3
Compute the structure tensor $C(x,y)$ by smoothing the squared derivatives with a Gaussian of width $\epsilon$.

In [ ]:
def structureTensor(im, sigma, epsilon):
    I, Ix, Iy = gaussianSmoothing(im, sigma)
    g_eps, _ = gaussian1DKernel(epsilon)
    Ix2 = cv2.sepFilter2D(Ix*Ix, -1, g_eps, g_eps)
    Iy2 = cv2.sepFilter2D(Iy*Iy, -1, g_eps, g_eps)
    Ixy = cv2.sepFilter2D(Ix*Iy, -1, g_eps, g_eps)
    return Ix2, Iy2, Ixy

A, B, C = structureTensor(test_im, 1.5, 2)
A.shape


## Exercise 6.4
Compute the Harris response $r(x,y) = \det(C) - k\,\mathrm{trace}(C)^2$. The secondary smoothing with $\epsilon$ is crucial; setting $\epsilon=0$ leaves the tensor unsmoothed and sensitive to noise.

In [ ]:
def harrisMeasure(im, sigma, epsilon, k=0.06):
    Ix2, Iy2, Ixy = structureTensor(im, sigma, epsilon)
    det = Ix2 * Iy2 - Ixy**2
    trace = Ix2 + Iy2
    r = det - k * trace**2
    return r

r = harrisMeasure(test_im, 1.5, 2, 0.06)
plt.imshow(r, cmap='gray')
plt.title('Harris response')
plt.axis('off')
plt.show()


## Exercise 6.5
Detect corners as local maxima of $r$ above a relative threshold $\tau$ using non-maximum suppression.

In [ ]:
def cornerDetector(im, sigma, epsilon, k, tau):
    r = harrisMeasure(im, sigma, epsilon, k)
    max_r = maximum_filter(r, size=3)
    corners = (r == max_r) & (r > tau * r.max())
    ys, xs = np.where(corners)
    return list(zip(xs, ys)), r

corners, r = cornerDetector(test_im, 1.5, 2, 0.06, 0.01)
fig, ax = plt.subplots()
ax.imshow(test_im, cmap='gray')
if corners:
    xs, ys = zip(*corners)
    ax.scatter(xs, ys, s=30, c='r')
ax.set_title('Detected corners')
ax.axis('off')
plt.show()


## Exercise 6.6
Apply the Canny edge detector to two images.

In [ ]:
def canny_edges(im, t1, t2):
    im_gray = color.rgb2gray(im) if im.ndim == 3 else im
    im_gray = (im_gray * 255).astype(np.uint8)
    edges = cv2.Canny(im_gray, t1, t2)
    return edges

im1 = data.camera()
im2 = data.astronaut()

edges1 = canny_edges(im1, 50, 150)
edges2 = canny_edges(im2, 50, 150)

fig, axs = plt.subplots(2,2, figsize=(8,8))
axs[0,0].imshow(im1, cmap='gray'); axs[0,0].set_title('Image 1'); axs[0,0].axis('off')
axs[0,1].imshow(edges1, cmap='gray'); axs[0,1].set_title('Canny'); axs[0,1].axis('off')
axs[1,0].imshow(color.rgb2gray(im2), cmap='gray'); axs[1,0].set_title('Image 2'); axs[1,0].axis('off')
axs[1,1].imshow(edges2, cmap='gray'); axs[1,1].set_title('Canny'); axs[1,1].axis('off')
plt.show()


## Exercise 6.7
Explore the effect of varying the hysteresis thresholds in the Canny edge detector.

In [ ]:
params = [(30,100), (50,150), (100,200)]
fig, axs = plt.subplots(1,3, figsize=(12,4))
for ax, (t1, t2) in zip(axs, params):
    ax.imshow(canny_edges(im2, t1, t2), cmap='gray')
    ax.set_title(f't1={t1}, t2={t2}')
    ax.axis('off')
plt.show()
